# AIFFELTHON -  [WONJIN]

## To do list

## 1. Pre-processing (Librosa, Scipy)

- Zero padding, 7secs
- Down sampling (sr= 16000(정인님팀 4000))
- Butter worth

## 2. Augmentation (Tensorflow-io + nlpaug ) on training data

- Shift Augmenation
- Speed Augmenation
- Pitch Augmentation


## 3. Feature Extraction

- Frequency masking
- Black Region Clipping

## 3. Modelling
- Resnet
- Transfer learning : Imagnet
: https://tensorflow.google.cn/tutorials/audio/transfer_learning_audio


## 4. Report


## 5. References

-  https://engineering.linecorp.com/ko/blog/voice-waveform-arbitrary-signal-to-noise-ratio-python/

- https://medium.com/analytics-vidhya/how-to-filter-noise-with-a-low-pass-filter-python-885223e5e9b7

- https://www.tensorflow.org/io/tutorials/audio



## 스마트 패딩 : 작업 중(-)

클래스가 같은 것 끼리 붙여서 7초로 만드는 것.



In [64]:
smart_root = '/content/drive/MyDrive/AIFFEL/smart_padding/'

In [63]:
# def getPureSample(raw_data,start,end,sr=16000):처음 패딩작업을 했을 때는 start와 end를 기준으로 각각을 짤라주었지만,
# 이번에 하는 작업은 두가지를 concat을 한것을 붙여서 7초로 만들 것이기 때문에 end를 7로 설정하여 짜르도록함.
def getPureSample(raw_data,end = 7*16000):
  if len(raw_data) > end:
    return raw_data[:end]
  else:
    return raw_data

In [60]:
# smart padding에 쓸 분류
normal1 = data2.loc[data2['crack_wheeze'] == 0]
print ('정상인 호흡음의 갯수 : ', len(normal1))

crackle1 = data2.loc[data2['crack_wheeze'] == 1]
print ('Crackle의 갯수 : ',len(crackle1))

wheezing1 =  data2.loc[data2['crack_wheeze'] == 2]
print ('Wheezing의 갯수 : ',len(wheezing1))

both1 = data2.loc[data2['crack_wheeze'] == 3]
print ('Crackle 및 wheezing 둘다 있는 경우의 갯수 : ',len(both1))

In [100]:
normal1.head()

,Unnamed: 0,start,end,pid,mode,filename,disease,crack_wheeze
4,4,17.671,19.541,160,mc,160_1b3_Ar_mc_AKGC417L_4.wav,COPD,0
5,5,0.109,4.766,160,mc,160_1b2_Tc_mc_AKGC417L_0.wav,COPD,0
11,11,17.671,19.541,160,mc,160_1b3_Pl_mc_AKGC417L_4.wav,COPD,0
12,12,1.862,5.718,160,mc,160_1b3_Tc_mc_AKGC417L_0.wav,COPD,0
16,16,17.671,19.541,160,mc,160_1b3_Tc_mc_AKGC417L_4.wav,COPD,0


In [73]:
# # 패딩용 np 전환 또는 파일 만들기

normal1_docs = []
normal1_audio_np = []
for i in normal1.filename:
  path3 = path2 + i
  normal1_docs.append(path3)

# for j in range(len(normal1_docs)):
#   normal_audio,_ = librosa.load(normal1_docs[j], sr = sr)
#   normal1_audio_np.append(normal_audio)


# crackle1_docs = []
# crackle1_audio_np = []
# for i in crackle1.filename:
#   path3 = path2 + i
#   crackle1_docs.append(path3)
# for j in range(len(crackle1_docs)):
#   crackle_audio,_ = librosa.load(crackle1_docs[j], sr = sr)
#   crackle1_audio_np.append(crackle_audio)


# wheezing1_docs = []
# wheezing1_audio_np = []
# for i in wheezing1.filename:
#   path3 = path2 + i
#   wheezing1_docs.append(path3)
# for j in range(len(wheezing1_docs)):
#   wheezing_audio,_ = librosa.load(wheezing1_docs[j], sr = sr)
#   wheezing1_audio_np.append(wheezing_audio)


# both1_docs = []
# both1_audio_np = []

# for i in both1.filename:
#   path3 = path2 + i
#   both1_docs.append(path3)

# for j in range(len(both1_docs)):
#   both_audio,_ = librosa.load(both1_docs[j], sr = sr)
#   both1_audio_np.append(both_audio)

In [107]:
normal2 = pd.DataFrame()
normal2['pid'] = normal1['pid']
normal2['start'] = normal1['start']
normal2['end'] = normal1['end']
normal2['mode'] = normal1['mode']
normal2['filename'] = normal1['filename']
normal2['crack_wheeze'] = normal1['crack_wheeze']
normal3=normal2.reset_index()
normal_2 = normal3.drop(['index'], axis=1)

normal_2.head()

,pid,start,end,mode,filename,crack_wheeze
0,160,17.671,19.541,mc,160_1b3_Ar_mc_AKGC417L_4.wav,0
1,160,0.109,4.766,mc,160_1b2_Tc_mc_AKGC417L_0.wav,0
2,160,17.671,19.541,mc,160_1b3_Pl_mc_AKGC417L_4.wav,0
3,160,1.862,5.718,mc,160_1b3_Tc_mc_AKGC417L_0.wav,0
4,160,17.671,19.541,mc,160_1b3_Tc_mc_AKGC417L_4.wav,0


In [248]:
test = []
maxLen = 7

def check_length(audio, maxLen):
  if len(audio)/16000 > maxLen:
    return test.append(audio)
  else:
    sample_p,_ = librosa.load(random.choice(normal1_docs))
    new = np.concatenate([audio,sample_p])
    return check_length(new, maxLen)

def audio_slicing(signal, max = 7 * sr):
  if len(signal) > max:
    return signal[:max]
  else:
    return signal    

      

In [273]:
## 혜선님 코드 응용

for idx,row in normal_2.iterrows():
  start = row['start']
  end = row['end']
  filename = row['filename'] 

  if idx == 3641:
    break

  elif normal_2.pid[idx] == normal_2.pid[idx+1]:
    sample_i,_ = librosa.load(normal1_docs[idx], sr=sr)
    sample_j,_ = librosa.load(normal1_docs[idx+1], sr=sr)
    
    new_sample = np.concatenate([sample_i, sample_j])
    check_data = len(new_sample)/sr

    if check_data > maxLen:
      new_sample = audio_slicing(new_sample)
      test.append(new_sample)

    else:
      check_length(new_sample, maxLen)  

  # elif normal_2.pid[idx] == 3642:
  #   break

  else:
    continue
    

  if len(test) % 100 ==0:
    print(len(test))

6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
